In [ ]:
from layup import routines
import numpy as np
import os

os.getcwd()

In [ ]:
def AD_vec(vec):
    x, y, z = vec
    Ax = z
    Ay = 0.0
    Az = -x
    A = np.sqrt(Ax * Ax + Ay * Ay + Az * Az)
    Ax /= A
    Ay /= A
    Az /= A
    Dx = -x * y
    Dy = x * x + z * z
    Dz = -z * y
    D = np.sqrt(Dx * Dx + Dy * Dy + Dz * Dz)
    Dx /= D
    Dy /= D
    Dz /= D
    return ([Ax, Ay, Az], [Dx, Dy, Dz])

In [ ]:
obs_list = []
with open("/Users/mholman/03666_out.txt") as file:
    for i, line in enumerate(file):
        objID, obsCode, mag, filt, jd_tdb, rho_x, rho_y, rho_z, xe, ye, ze, ast_unc = line.rstrip().split()
        jd_tdb = float(jd_tdb)
        rho_x, rho_y, rho_z = float(rho_x), float(rho_y), float(rho_z)
        xe, ye, ze = float(xe), float(ye), float(ze)
        ast_unc = float(ast_unc) / 206265
        Av, Dv = AD_vec((rho_x, rho_y, rho_z))
        rho = [rho_x, rho_y, rho_z]
        re = [xe, ye, ze]
        ve = [0.0, 0.0, 0.0]
        obs = routines.Observation(jd_tdb, re, ve, rho, Av, Dv, ast_unc, ast_unc)
        obs_list.append(obs)

In [ ]:
GMtotal = 0.0002963092748799319
AU_M = 149597870700
SPEED_OF_LIGHT = 2.99792458e8 * 86400.0 / AU_M

In [ ]:
solns = routines.gauss(GMtotal, obs_list[4300], obs_list[4308], obs_list[4311], 0.0001, SPEED_OF_LIGHT)

In [ ]:
assist_ephem = routines.get_ephem("/Users/mholman/Library/Caches/layup/")

In [ ]:
x = routines.run_from_vector_with_ic(assist_ephem, solns[0], obs_list[-40:])

In [ ]:
x = routines.run_from_vector_with_ic(assist_ephem, x, obs_list)

In [ ]:
x = routines.run_from_vector(assist_ephem, obs_list)

In [ ]:
x.dof, x.niter, x.state, x.csq, x.epoch